In [1]:
import torch
from torch.autograd import Variable
from RNN import *
import numpy as np
import deepdish as dd
import matplotlib.pyplot as plt
import os
import cv2

### use gpu

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
use_gpu = torch.cuda.is_available()

### load data

In [3]:
combo_all = dd.io.load('20classes_combo_padzero_1000features.h5') # utilize pad ground truth

In [4]:
gt_all = dd.io.load('20classes_normal_bboxes_all_sqrtwh_list.h5')

In [5]:
feature = Variable(combo_all[0][0])

In [6]:
class_file = '../routine_generate_vot2017_train/vot2017/list_20classes.txt'

In [7]:
CLASSES = [line.rstrip('\n') for line in open(class_file)]

### set hyperparameters

In [8]:
#n_data = 50                   # 20 data needs two process
n_features = feature.size(0)              # YOLO output plus padding ground truth location or zeros
hidden_size = input_size = 500
n_batch = 1                   # number of batch
n_layers = 1      
time_steps = 10
n_classes = 2

In [9]:
n_features

1004

### load model

In [10]:
model = RNN_module(n_features, input_size, hidden_size, n_batch)
model.load_state_dict(torch.load('./saved_models/DRLT_2classes_1000features.pth'))
model.eval()
if use_gpu:
    model.cuda()

-----initializing RNN-----


FileNotFoundError: [Errno 2] No such file or directory: './saved_models/DRLT_2classes_1000features.pth'

### get prediction of location

In [ ]:
def save_results(image, pred_location, result_folder, class_name, jpgname):
    x, y, width, height = int(pred_location[0]), int(pred_location[1]), int(pred_location[2]), int(pred_location[3])
    left_up, right_bottom = (x-width, y-height), (x+width, y+height)
    cv2.rectangle(image,left_up,right_bottom,(0,255,0),2)
    cv2.putText(image,class_name,left_up,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1,cv2.LINE_AA)
    cv2.imwrite(os.path.join(result_folder, jpgname),image)

In [ ]:
def make_prediction(model, img_folder, result_folder, loop_begin, CLASSES, class_i):
    loop_begin_2 = loop_begin
    
    test_img = plt.imread(os.path.join(img_folder, '00000001.jpg'))
    height,width,_ = test_img.shape
    
    h_state = Variable(torch.randn(n_layers, n_batch, hidden_size)).cuda()
    c_state = Variable(torch.randn(n_layers, n_batch, hidden_size)).cuda()
    
    before = 5
    for loop in range(before):
        combo = Variable(combo_all[class_i])
        input = combo[loop_begin-before:loop_begin-before + 1]
        input = input.cuda()

        h_state, c_state = model(input, h_state, c_state)
        pred_location = h_state[0,0][-4:].cpu().data.numpy() * np.array([width, height, np.sqrt(width), np.sqrt(height)])
        pred_location[2:] = pred_location[2:]**2
        #plt.imshow(test_img)
        #plt.plot(pred_location[0], pred_location[1], 'ro')
        #plt.plot()
        #plt.show()
        loop_begin += 1
    
    for loop in range(10):
        combo = Variable(combo_all[class_i])
        input = combo[loop_begin_2:loop_begin_2 + 1]
        input = input.cuda()

        h_state, c_state = model(input, h_state, c_state)
        pred_location = h_state[0,0][-4:].cpu().data.numpy() * np.array([width, height, np.sqrt(width), np.sqrt(height)])
        pred_location[2:] = pred_location[2:]**2
        
        jpgname = '{0:08}'.format(loop_begin_2) + '.jpg'
        print(jpgname)
        img = plt.imread(os.path.join(img_folder, jpgname))
        class_name = CLASSES[class_i]
        save_results(img, pred_location, result_folder, class_name, jpgname)
        #plt.imshow(img)
        #plt.plot(pred_location[0], pred_location[1], 'ro')
        #plt.plot()
        #plt.show()
        #print(pred_location)
        #break
        loop_begin_2 += 1
    return pred_location

In [ ]:
# test class_i = 1, bag
class_i = 0
img_folder = '../routine_generate_vot2017_train/vot2017/' + CLASSES[class_i]
result_folder = './result_imgs/'
loop_begin = gt_all[class_i].shape[0] // 3 // time_steps * time_steps + 1

In [ ]:
pred_location = make_prediction(model, img_folder, result_folder, loop_begin, CLASSES, class_i)

### save model

In [ ]:
#torch.save(model.state_dict(),'./saved_models/DRLT_2classes_1000features_2ndreward.pth')